In [1]:
import numpy as np
import torch
from torch import Tensor
import torch.nn as nn
from torch import matmul
from torch import tanh
from torch import Tensor
from torch.nn.functional import relu
from torch.nn.functional import leaky_relu
from torch import sigmoid
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64)

In [2]:
import os
os.chdir('/Volumes/My Passport/eem20/')

In [23]:
# Gaussian Neural Density Estimator (2-layer version)
# takes some inputs (turbine properties and weather forecasts) and outputs a mean and (log) variance describing a Gaussian distribution for the predicted power output
class GaussianNDE(nn.Module):
    
    # init
    def __init__(self, n_turbine_features, n_weather_features, n_units=[64,64], activation=leaky_relu, mean_init=0, logvar_init=1.):
        
        super(GaussianNDE, self).__init__()
        
        # architecture: (n_turbines, n_weather) -> hidden layers (n_units) -> 2 outputs (mean and log variance)
        self.n_turbine_features = n_turbine_features
        self.n_weather_features = n_weather_features
        self.n_units = n_units
        
        # network weights
        self.W1t = nn.Parameter(torch.Tensor(self.n_turbine_features, self.n_units[0]).normal_(mean=0, std=1e-5), requires_grad=True)
        self.W1w = nn.Parameter(torch.Tensor(self.n_weather_features, self.n_units[0]).normal_(mean=0, std=1e-5), requires_grad=True)
        self.W2 = nn.Parameter(torch.Tensor(self.n_units[0], self.n_units[1]).normal_(mean=0, std=1e-5), requires_grad=True)
        self.W3 = nn.Parameter(torch.Tensor(self.n_units[1], 2).normal_(mean=0, std=1e-5), requires_grad=True)
        
        # network biases
        self.b1 = nn.Parameter(torch.Tensor(self.n_units[0]).normal_(mean=0, std=1e-5), requires_grad=True)
        self.b2 = nn.Parameter(torch.Tensor(self.n_units[1]).normal_(mean=0, std=1e-5), requires_grad=True)
        self.b3 = nn.Parameter(torch.Tensor(2).normal_(mean=0, std=1e-5), requires_grad=True)
        
        # initialize the bias of the output layer
        self.b3[0].data.fill_(mean_init)
        self.b3[1].data.fill_(logvar_init)

        # activation
        self.activation = activation
    
    # forward pass through the network
    def forward(self, turbine_features, weather_features):
        
        # forward pass through two hidden layers (including split input layer)
        output = torch.matmul(self.activation(torch.matmul(self.activation(torch.matmul(turbine_features, self.W1t) + torch.matmul(weather_features, self.W1w) + self.b1), self.W2) + self.b2), self.W3) + self.b3
        
        # split the output layer into two outputs: mean nd log variance (log_variance)
        mean, log_variance = torch.split(output, 1, dim=-1)
               
        return mean, log_variance
    
    # batched forward pass
    def batched_forward(self, turbine_features, weather_features):
        
        # forward pass through two hidden layers (including split input layer)
        output = torch.einsum("ijk,kl->ijl", self.activation(torch.einsum("ijk,kl->ijl", self.activation(torch.einsum("ijk,kl->ijl", turbine_features, self.W1t) + torch.einsum("ijk,kl->ijl", weather_features, self.W1w) + self.b1), self.W2) + self.b2), self.W3) + self.b3
        
        # split the output layer into two outputs: mean nd log variance (log_variance)
        mean, log_variance = torch.split(output, 1, dim=-1)
               
        return mean, log_variance

# same as above but with generic network depth
class GaussianNDE_Deep(nn.Module):
    
    # init
    def __init__(self, n_inputs, n_units=[64,64], activation=leaky_relu):
        
        super(GaussianNDE, self).__init__()
        
        # architecture: n_inputs -> hidden layers (n_units) -> 2 outputs (mean and variance)
        self.n_inputs = n_inputs # n_inputs = number of turbine features + number of weather features (ie., total length of input feature vector)
        self.architecture = [self.n_inputs] + n_units + [2]
        self.n_layers = len(self.architecture) - 1
        
        # network weights and biases
        self.W = nn.ParameterList([nn.Parameter(torch.Tensor(self.architecture[i], self.architecture[i+1]), requires_grad=True) for i in range(self.n_layers)])
        self.b = nn.ParameterList([nn.Parameter(torch.Tensor(self.architecture[i+1]), requires_grad=True) for i in range(self.n_layers)])
        
        # activation
        self.activation = activation
    
    # forward pass through the network
    def forward(self, turbine_features, weather_features):
        
        # initialize input layer: concatenate the turbine features and weather features together into a single input vector
        layers = [torch.cat((turbine_features, weather_features), -1)]
        
        # implement network
        for i in range(self.n_layers):
            layers.append(self.activation(matmul(layers[-1], self.W[i]) + self.b[i]))
        
        # split the output layer into two outputs: mean nd log variance (log_variance)
        mean, log_variance = torch.split(layers[-1], 1, dim=-1)
               
        return mean, log_variance

In [24]:
# import your data here

# turbine features
turbines_SE1 = torch.tensor(np.load('data_prepared/turbines_SE1.npy'))
turbines_SE2 = torch.tensor(np.load('data_prepared/turbines_SE2.npy'))
turbines_SE3 = torch.tensor(np.load('data_prepared/turbines_SE3.npy'))
turbines_SE4 = torch.tensor(np.load('data_prepared/turbines_SE4.npy'))
turbine_features_shift = torch.tensor(np.load('data_prepared/turbine_features_shift.npy'))
turbine_features_scale = torch.tensor(np.load('data_prepared/turbine_features_scale.npy'))
turbine_features = [(turbines_SE1 - turbine_features_shift)/turbine_features_scale, 
                    (turbines_SE2 - turbine_features_shift)/turbine_features_scale, 
                    (turbines_SE3 - turbine_features_shift)/turbine_features_scale, 
                    (turbines_SE4 - turbine_features_shift)/turbine_features_scale]

# weather
weather_SE1 = torch.tensor(np.load('data_prepared/weather_at_turbine_locations_SE1.npy'))
weather_SE2 = torch.tensor(np.load('data_prepared/weather_at_turbine_locations_SE2.npy'))
weather_SE3 = torch.tensor(np.load('data_prepared/weather_at_turbine_locations_SE3.npy'))
weather_SE4 = torch.tensor(np.load('data_prepared/weather_at_turbine_locations_SE4.npy'))
weather_features_shift = torch.tensor(np.load('data_prepared/weather_features_shift.npy'))
weather_features_scale = torch.tensor(np.load('data_prepared/weather_features_scale.npy'))
weather = [(weather_SE1 - weather_features_shift)/weather_features_scale, 
           (weather_SE2 - weather_features_shift)/weather_features_scale, 
           (weather_SE3 - weather_features_shift)/weather_features_scale, 
           (weather_SE4 - weather_features_shift)/weather_features_scale]

# installed or not?
installed_SE1 = torch.tensor(np.load('data_prepared/installed_SE1.npy'))
installed_SE2 = torch.tensor(np.load('data_prepared/installed_SE2.npy'))
installed_SE3 = torch.tensor(np.load('data_prepared/installed_SE3.npy'))
installed_SE4 = torch.tensor(np.load('data_prepared/installed_SE4.npy'))
installed = [installed_SE1, installed_SE2, installed_SE3, installed_SE4]

# sector energy
sector_power_SE1 = torch.tensor(np.load('data_prepared/sector_power_SE1.npy'))
sector_power_SE2 = torch.tensor(np.load('data_prepared/sector_power_SE2.npy'))
sector_power_SE3 = torch.tensor(np.load('data_prepared/sector_power_SE3.npy'))
sector_power_SE4 = torch.tensor(np.load('data_prepared/sector_power_SE4.npy'))
sector_power = [sector_power_SE1, sector_power_SE2, sector_power_SE3, sector_power_SE4]

# number of time-steps, sectors, turbine features and weather features
n_times = weather_SE1.shape[0]
n_turbine_features = turbine_features[0].shape[-1]
n_weather_features = weather[0].shape[-1]
n_sectors = 4

# mean and std of power for init
mean_init = torch.mean(sector_power[0])/len(turbines_SE1)
logvar_init = torch.log( torch.var(sector_power[0])/len(turbines_SE1) )

In [29]:
weather[0].size()

torch.Size([10152, 472, 12])

In [25]:
# initialize a neural network model for the turbine power outputs
Model = GaussianNDE(n_turbine_features=n_turbine_features, 
                    n_weather_features=n_weather_features, 
                    n_units=[64, 64], 
                    activation=leaky_relu,
                    mean_init=mean_init,
                    logvar_init=logvar_init)

In [26]:
# training parameters
learning_rate = 1e-2
epochs = 20
adam_regularisation = 0.0

# optimizer
optimiser = torch.optim.Adam(Model.parameters(), lr=learning_rate, weight_decay=adam_regularisation)

# sector
sector = 2

In [28]:
# MSE training loop

# progress bar
pbar = tqdm.tqdm_notebook(total=epochs, desc = "Training")
pbar.set_postfix(ordered_dict={"loss":0}, refresh=True)

# holder for training loss
training_loss = []

# loop over epochs
for epoch in range(epochs):
    
    # construct batches
    batches = np.random.permutation(np.arange(10000)).reshape(10, 1000)
    
    # loop over time batches
    for t in range(len(batches)):
        
        batch = batches[t]
        
        # initialize the loss
        #loss = torch.tensor(0.0, requires_grad=True)
        
        # loop over sectors
        for s in [sector]:

                # calculate the predicted means and variances for those turbines in that sector, at that time
                # note: turbine features is n_turbines x n_features, and weather is n_times x n_turbines x n_features
                means, log_variances = Model(turbine_features[s], weather[s][batch,:,:])

                # calculate the predicted mean and variance for the whole sector
                # note that we multiply by "installed", which will kill any terms coming from turbines that were not installed at this time
                # installed is n_times x n_turbines
                sector_mean = torch.sum(means * installed[s][batch,:,:], axis=1)
                sector_variance = torch.sum(torch.exp(log_variances) * installed[s][batch,:,:], axis=1)

                # calculate log_likelihoods
                #neg_log_likelihood = 0.5*(sector_power[s][batch,:] - sector_mean).pow(2)/sector_variance + 0.5*torch.log(sector_variance)
                neg_log_likelihood = torch.sqrt(torch.mean((sector_power[s][batch,:] - sector_mean).pow(2)))
                
                # add to the loss: this is the negative log-likelihood of a gaussian with the predicted sector mean and variance from above
                #loss = loss + torch.sum(neg_log_likelihood)
                loss = neg_log_likelihood

        # backpropagation step (ie., gradient descent step)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        pbar.set_postfix(ordered_dict={"loss":loss.data.clone().numpy().flatten()[0]}, refresh=True)
        
    # save training loss
    training_loss.append(loss.data.clone().numpy().flatten()[0])

    # update progress bar
    pbar.update(1)
    pbar.set_postfix(ordered_dict={"loss":training_loss[-1]}, refresh=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


KeyboardInterrupt: 

In [ ]:
# make predictions for SE1

t0 = 0
t1 = 1024
mean, logvar = Model(turbine_features[sector], weather[sector][t0:t1,:,:])
sector_mean = torch.sum(mean * installed[sector][t0:t1,:,:], axis=1).detach().numpy()[:,0]
sector_std = torch.sqrt(torch.sum(torch.exp(logvar) * installed[sector][t0:t1,:,:], axis=1)).detach().numpy()[:,0]

plt.fill_between(np.arange(t0, t1), sector_mean - sector_std, sector_mean + sector_std, alpha=0.1)
plt.plot(np.arange(t0, t1), sector_mean, color = 'red', label='model')
plt.plot(np.arange(t0, t1), sector_power[sector][t0:t1], color = 'blue', label='data')
plt.legend(frameon=False)
plt.show()

plt.plot(np.arange(t0, t1), sector_mean-sector_power[sector][t0:t1].detach().numpy()[:,0], label='residuals')
plt.legend(frameon=False)
plt.show()

In [ ]:
# std deviation on predictions
mean, logvar = Model(turbine_features[sector], weather[sector])
p = sector_power[sector].detach().numpy()[:,0]
sector_mean = torch.clamp(torch.sum(mean * installed[sector], axis=1), min(p), max(p)).detach().numpy()[:,0]
error_std = np.std(sector_mean - p)

In [ ]:
print(error_std)
print(np.std(p))

In [ ]:
# Make predictions

# load in data
# turbine features
turbines_SE1_v = torch.tensor(np.load('round2_data_prepared/turbines_SE1.npy'))
turbines_SE2_v = torch.tensor(np.load('round2_data_prepared/turbines_SE2.npy'))
turbines_SE3_v = torch.tensor(np.load('round2_data_prepared/turbines_SE3.npy'))
turbines_SE4_v = torch.tensor(np.load('round2_data_prepared/turbines_SE4.npy'))
turbine_features_shift = torch.tensor(np.load('data_prepared/turbine_features_shift.npy'))
turbine_features_scale = torch.tensor(np.load('data_prepared/turbine_features_scale.npy'))
turbine_features_v = [(turbines_SE1_v - turbine_features_shift)/turbine_features_scale, 
                    (turbines_SE2_v - turbine_features_shift)/turbine_features_scale, 
                    (turbines_SE3_v - turbine_features_shift)/turbine_features_scale, 
                    (turbines_SE4_v - turbine_features_shift)/turbine_features_scale]

# weather
weather_SE1_v = torch.tensor(np.load('round2_data_prepared/weather_at_turbine_locations_SE1.npy'))
weather_SE2_v = torch.tensor(np.load('round2_data_prepared/weather_at_turbine_locations_SE2.npy'))
weather_SE3_v = torch.tensor(np.load('round2_data_prepared/weather_at_turbine_locations_SE3.npy'))
weather_SE4_v = torch.tensor(np.load('round2_data_prepared/weather_at_turbine_locations_SE4.npy'))
weather_features_shift = torch.tensor(np.load('data_prepared/weather_features_shift.npy'))
weather_features_scale = torch.tensor(np.load('data_prepared/weather_features_scale.npy'))
weather_v = [(weather_SE1_v - weather_features_shift)/weather_features_scale, 
           (weather_SE2_v - weather_features_shift)/weather_features_scale, 
           (weather_SE3_v - weather_features_shift)/weather_features_scale, 
           (weather_SE4_v - weather_features_shift)/weather_features_scale]

# installed or not?
installed_SE1_v = torch.tensor(np.load('round2_data_prepared/installed_SE1.npy'))
installed_SE2_v = torch.tensor(np.load('round2_data_prepared/installed_SE2.npy'))
installed_SE3_v = torch.tensor(np.load('round2_data_prepared/installed_SE3.npy'))
installed_SE4_v = torch.tensor(np.load('round2_data_prepared/installed_SE4.npy'))
installed_v = [installed_SE1_v, installed_SE2_v, installed_SE3_v, installed_SE4_v]

# sector energy
sector_power_SE1_v = torch.tensor(np.load('round2_data_prepared/sector_power_SE1.npy'))
sector_power_SE2_v = torch.tensor(np.load('round2_data_prepared/sector_power_SE2.npy'))
sector_power_SE3_v = torch.tensor(np.load('round2_data_prepared/sector_power_SE3.npy'))
sector_power_SE4_v = torch.tensor(np.load('round2_data_prepared/sector_power_SE4.npy'))
sector_power_v = [sector_power_SE1_v, sector_power_SE2_v, sector_power_SE3_v, sector_power_SE4_v]

In [ ]:
# construct percentiles
means, _ = Model(turbine_features_v[sector], weather_v[sector])
sector_mean = torch.sum(means * installed_v[sector], axis=1).detach().numpy()[:,0]
maximum = np.max(sector_power[sector].detach().numpy())

percentiles = np.zeros((len(weather_v[sector]), 9))

for i in range(len(weather_v[sector])):
    
    samples = np.random.normal(sector_mean[i], error_std, 10000)
    samples[(samples > 0) * (samples < maximum)]
    percentiles[i,:] = np.percentile(samples, [10, 20, 30, 40, 50, 60, 70, 80, 90])

In [ ]:
np.save('round2_percentiles/percentiles_SE{}.npy'.format(sector+1), percentiles)